# Wstęp do Sztucznej Inteligencji - rok akademicki 2018/2019

Przed rozpoczęciem pracy z notatnikiem zmień jego nazwę zgodnie z wzorem: `{NrAlbumu}_{Nazwisko}_{Imie}_{PoprzedniaNazwa}`.

Przed wysłaniem notatnika upewnij się, że rozwiązałeś wszystkie zadania/ćwiczenia, w szczególności, że uzupełniłeś wszystkie pola `YOUR CODE HERE` oraz `YOUR ANSWER HERE`.

## Temat: Algorytmy genetyczne - Lab 4 - Problem plecakowy - Zadania (obowiązkowe)
Zapoznaj się z treścią niniejszego notatnika czytając i wykonując go komórka po komórce. Wykonaj napotkane zadania/ćwiczenia.

### Problem plecakowy
Problem plecakowy to problem optymalizacji dyskretnej. W swojej najprostszej wersji może być sformułowany następująco.

Spośród `N` dostępnych przedmiotów możmy zabrać taką ich liczbę by suma ich wag nie przekraczała pewnej dopuszczalnej wartości (maksymalne obciążenie plecaka). Każdy przedmiot, oprócz wagi, ma przypisaną swoją wartość. 

Które przedmioty zbrać, aby ich sumaryczna wartość była jak największa, ale ich sumaryczna waga nie przekraczała dopuszczalnej maksymalnej wagi plecaka?

Mamy zatem do czynienia z problemem optymalizacji z ograniczeniami.

Przeczytaj więcej o problemie plecakowym:

https://pl.wikipedia.org/wiki/Problem_plecakowy

### Prosty generator problemu plecakowego (0-1 knapsack problem)

Generujemy listę przedmiotów, z losowymi wagami oraz wartościami z podanych przedziałów.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
#%matplotlib notebook

#wmin - minimalna waga przedmiotu
#wmax - maksymalna waga przedmiotu
#vmin - minimalna wartość przedmiotu
#vmax - maksymalna wartość przedmiotu
#items_num - liczba dostępnych przedmiotów
def generate_problem(wmin, wmax, vmin, vmax, items_num):
    w = np.random.randint(wmin, wmax, size=items_num)  #weight
    v = np.random.randint(vmin, vmax, size=items_num)  #values
    return w, v

Przykładowy problem plecakowy (zwróć uwagę, że możemy zagwarantować generowanie za każdym razem tego samego problemu poprzez ustawienie ziarna generatora liczb losowych).

In [ ]:
num = 50  # liczba przedmiotów
wmin = 1  # minimlana waga
wmax = 100  # maksymalna waga
vmin = 1  # minimalna wartosc
vmax = 100  # maksymalna wartosc
knapsack_perc = 0.5  # pojemnosc plecaka jako procent sumy wag wszystkich przedmiotow

# ustawienie ziarna
np.random.seed(1111)

w, v = generate_problem(wmin, wmax, vmin, vmax, num)  # w - wagi, v - wartosci
Wall = w.sum()
Vall = v.sum()
W = int(knapsack_perc * Wall) # pojemnosc plecaka

print('Problem plecakowy:')
print('pojemnosc plecaka:', W)
print('wagi:',w, 'suma:', Wall)
print('wartosci:',v, 'suma:', Vall)

## Zadanie 1 (1pkt.) - rozwiązanie metodą brute force

Mamy problem plecakowy ze 100 przedmiotami. Chcemy sprawdzić każde możliwe rozwiązanie. Jeśli w ciągu sekundy moglibysmy sprawdzić miliard rozwiązań, to ile lat by to trwało? Przyjmij, że rok ma 365 dni.

Otrzymaną liczbę lat przypisz do zmiennej o nazwie `liczba_lat`. Wynik zaokrąglij do pełnych lat w dół.

In [2]:
# YOUR CODE HERE
liczba_lat = int(2**100 / 10**9 / 365 / 24 / 3600)
print('Zajmie to: {} lat'.format(liczba_lat))

Zajmie to: 40196936841331 lat


In [ ]:
#Komórka z autotestem

## Kodowanie rozwiązania

Potencjalne rozwiązanie problemu plecakowego można zakodować jako wektor `0`/`1`, gdzie `1` oznacza wybranie przedmiotu do plecaka.

Warto skorzystać z kodowania `True`/`False` i tablic `numpy`, gdyż ułatwi to obliczenia (macierze True/False mogą służyć do indeksowania innych macierzy).

Przykładowo:

In [ ]:
num = w.shape[0]
sol = np.random.randint(0, 2, size=num, dtype=np.bool)  # True oznacza, ze przedmiot jest wybrany

print('Rozwiązanie:', sol)
print('Lista wybranych przedmiotow:', np.arange(num)[sol])
print('Suma wag:', w[sol].sum())
print('Suma wartosci:', v[sol].sum())

## Rozwiązania dopuszczalne i niedopuszczalne - procedura naprawcza

Jeśli suma wag przekracza pojemność plecaka, to rozwiązanie jest rozwiązaniem niedopuszczalnym i jest całkowicie nieprzydatne.

Rozwiązanie niedopuszczalne można poddać procedurze naprawczej. Przykładowo, poniższa funkcja usuwa przedmioty z plecaka, dopóki nie powstanie rozwiązanie dopuszczalne.

In [ ]:
# Usuwa losowe przedmioty, aż rozwiązanie stanie się dopuszczalne
# Jesli rowziazanie jest dopuszczlane, nie zostanie zmienione
def correct_solution(w, v, W, sol):
    num = w.shape[0]
    while w[sol].sum() > W:
        indx = np.random.randint(num)
        while sol[indx%num] == False:
            indx = indx + 1
        sol[indx%num] = False

#### Przykładowa naprawa:

In [ ]:
correct_solution(w, v, W, sol)

print('Rozwiązanie:', sol)
print('Lista wybranych przedmiotow:', np.arange(num)[sol])
print('Suma wag:', w[sol].sum())
print('Suma wartosci:', v[sol].sum())

## Losowe, dopuszczalne rozwiązania

Bazując na powyższej procedurze, można zdefiniować funkcję generującą losowe, ale zawsze dopuszczalne rozwiązania.

In [18]:
def get_random_solution(w, v, W):
    num = w.shape[0]
    sol = np.random.randint(0,2, size=num, dtype=np.bool)  # 1 / True oznacza, ze przedmiot jest wybrany
    _V = np.sum(v[sol])
    _W = np.sum(w[sol])
    if _W > W:
        correct_solution(w,v,W,sol)
        _V = np.sum(v[sol])
        _W = np.sum(w[sol])
    return sol, _W, _V

#### Przykład użycia:

In [ ]:
sol = get_random_solution(w,v,W)

print('Rozwiązanie:', sol[0])
print('Lista wybranych przedmiotow:', np.arange(num)[sol[0]])
print('Suma wag:', sol[1])
print('Suma wartosci:', sol[2])

## Random search

Powyższe funkcje można przełożyć na prostą heurystykę przeszukiwania losowego. Generujemy losowe (ale dopuszczalne) rozwiązania przez zadaną liczbę iteracji i zapamiętujemy najlepsze. Dodatkowo, zapisujemy oceny rozwiązań (wartość wszystkich przedmiotów w plecaku) by przedstawić je na wykresie.

In [20]:
def search_random(w,v,W,iters):
    best_sol, best_W, best_V = get_random_solution(w,v,W)
    v_all = [best_V]
    v_best = [best_V]
    for i in range(iters):
        sol, _W, _V = get_random_solution(w,v,W)
        if best_V < _V:
            best_sol, best_W, best_V = sol, _W, _V
        v_all.append(_V)
        v_best.append(best_V)
#     plt.figure()
#     plt.plot(v_all)
#     plt.plot(v_best)
#     plt.show()
    return best_sol, best_W, best_V, v_all, v_best

#### Przykładowe uruchomienie:

In [ ]:
sol_random_search = search_random(w, v, W, 1000)

print('Najlepsze rozwiązanie:',sol_random_search[0])
print('Przedmioty:',np.arange(num)[sol_random_search[0]])
print('Suma wag:', sol_random_search[1])
print('Suma wartosci:', sol_random_search[2])

## Greedy search - procedura optymalizacji zachłannej

Problemem w powyższym podejściu jest fakt, że kolejne rozwiązania nie korzystają z uzyskanej już wiedzy o najlepszych do tej pory rozwiązaniach. Zatem inna prosta heurytyka polega na wystartowaniu z losowego rozwiązania, a następnie modyfikowaniu go poprzez losowe dodawanie przedmiotu do plecaka. Jeśli zmiana (po ewentualnej naprawie) wprowadza poprawę, pozostajemy przy takim rozwiązaniu, jeśli następuje pogorszenie, odrzucamy takie rozwiązanie i ponawiamy próbę.

Taka procedura jest przykładem optymalizacji zachłannej.

In [21]:
def search_greedy_improvement(w, v, W, iters):
    best_sol, best_W, best_V = get_random_solution(w,v,W)
    v_all = [best_V]
    v_best = [best_V]
    num = w.shape[0]
    for i in range(iters):
        sol = best_sol.copy()
        #set random 0 bit to 1
        indx = np.random.randint(num)
        while sol[indx%num] == True:
            indx = indx + 1
        sol[indx%num] = True
        #correct if needed
        if w[sol].sum() > W:
            correct_solution(w,v,W,sol)
        _V = v[sol].sum()
        _W = w[sol].sum()
        if best_V < _V:
            best_sol, best_W, best_V = sol.copy(), _W, _V
        v_all.append(_V)
        v_best.append(best_V)
#     plt.figure()
#     plt.plot(v_all)
#     plt.plot(v_best)
#     plt.show()
    return best_sol, best_W, best_V, v_all, v_best 

#### Przykładowe uruchomienie:

In [ ]:
sol_greedy = search_greedy_improvement(w,v,W,1000)

print('Najlepsze rozwiązanie:',sol_greedy[0])
print('Przedmioty:',np.arange(num)[sol_greedy[0]])
print('Suma wag:', sol_greedy[1])
print('Suma wartosci:', sol_greedy[2])

## Dwie inne proste heurystyki

### Najpierw najbardziej wartościowe

Jak dobre rozwiązanie dostaniemy, jeśli do plecaka pakować będziemy najpierw najbardziej wartościowe przedmioty (o ile się zmieszczą)? Pomysł ten jest zaimplementowany w poniższej funkcji.


In [15]:
#Pakuje najpierw najbardziej wartościowe przedmioty
def get_value_first(w, v, W):
    ii = np.argsort(-v)
    num = w.shape[0]
    sol = np.repeat(False, num)
    _W = 0
    for i in range(num):
        if _W + w[ii[i]] <= W:
            sol[ii[i]] = True
            _W = _W + w[ii[i]]
    _V = v[sol].sum()
    return sol, _W, _V

#### Przykładowe uruchomienie:

In [ ]:
sol_value_first = get_value_first(w, v, W)

print('Najlepsze rozwiązanie:',sol_value_first[0])
print('Przedmioty:',np.arange(num)[sol_value_first[0]])
print('Suma wag:', sol_value_first[1])
print('Suma wartosci:', sol_value_first[2])

### Najpierw te o najlepszym stosunku wartości do wagi

Inny pomysł to pakowanie najpierw przedmiotów o najlepszym stosunku wartości do wagi.


In [16]:
def get_ratio_first(w, v, W):
    ii = np.argsort(-v/w) #stosunek wartosci do wagi
    num = w.shape[0]
    sol = np.repeat(False, num)
    _W = 0
    for i in range(num):
        if _W + w[ii[i]] <= W:
            sol[ii[i]] = True
            _W = _W + w[ii[i]]
    _V = v[sol].sum()
    return sol, _W, _V

#### Przykładowe uruchomienie:

In [ ]:
sol_ratio_first = get_ratio_first(w, v, W)

print('Najlepsze rozwiązanie:',sol_ratio_first[0])
print('Przedmioty:',np.arange(num)[sol_ratio_first[0]])
print('Suma wag:', sol_ratio_first[1])
print('Suma wartosci:', sol_ratio_first[2])

## Zadanie 2 (3 pkt.)

Opracowane być mogą inne procedury naprawcze. Przykładowo, dla rozwiązania niedopuszczalengo, zamiast zmieniać losowe bity True na False, jak w funkcji `correct_solution`, można usuwać najpierw te przedmioty, które mają najgorszy stosunek wartości do wagi. Zaimplementuj taką procedurę. 

Porównaj jej działanie z funkcją `correct_solution` w metodach RandomSearch oraz GreedySearch. Przedstaw uśrednione wyniki (co najmniej 10 uruchomień) i wnioski.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
#%matplotlib notebook

#wmin - minimalna waga przedmiotu
#wmax - maksymalna waga przedmiotu
#vmin - minimalna wartość przedmiotu
#vmax - maksymalna wartość przedmiotu
#items_num - liczba dostępnych przedmiotów
def generate_problem(wmin, wmax, vmin, vmax, items_num):
    w = np.random.randint(wmin, wmax, size=items_num)  #weight
    v = np.random.randint(vmin, vmax, size=items_num)  #values
    return w, v

def correct_solution(w, v, W, sol):
    num = w.shape[0]
    while w[sol].sum() > W:
        indx = np.random.randint(num)
        while sol[indx%num] == False:
            indx = indx + 1
        sol[indx%num] = False

def correct_solution2(w, v, W, sol):
    ratio = v/w
    num = w.shape[0]
    while w[sol].sum() > W:
        while True:
            indx = np.argmin(ratio)
            if sol[indx] == False:
                ratio[indx] = 10000000
            else:
                sol[indx] = False
                break

def get_random_solution(w, v, W):
    num = w.shape[0]
    sol = np.random.randint(0,2, size=num, dtype=np.bool)  # 1 / True oznacza, ze przedmiot jest wybrany
    _V = np.sum(v[sol])
    _W = np.sum(w[sol])
    if _W > W:
        correct_solution(w,v,W,sol)
        _V = np.sum(v[sol])
        _W = np.sum(w[sol])
    return sol, _W, _V

def get_random_solution_improved(w, v, W):
    num = w.shape[0]
    sol = np.random.randint(0,2, size=num, dtype=np.bool)  # 1 / True oznacza, ze przedmiot jest wybrany
    _V = np.sum(v[sol])
    _W = np.sum(w[sol])
    if _W > W:
        correct_solution2(w,v,W,sol)
        _V = np.sum(v[sol])
        _W = np.sum(w[sol])
    return sol, _W, _V

def search_random(w,v,W,iters):
    best_sol, best_W, best_V = get_random_solution(w,v,W)
    v_all = [best_V]
    v_best = [best_V]
    for i in range(iters):
        sol, _W, _V = get_random_solution(w,v,W)
        if best_V < _V:
            best_sol, best_W, best_V = sol, _W, _V
        v_all.append(_V)
        v_best.append(best_V)
    # plt.figure()
    # plt.plot(v_all)
    # plt.plot(v_best)
    # plt.show()
    return best_sol, best_W, best_V, v_all, v_best

def search_random_improved(w,v,W,iters):
    best_sol, best_W, best_V = get_random_solution_improved(w,v,W)
    v_all = [best_V]
    v_best = [best_V]
    for i in range(iters):
        sol, _W, _V = get_random_solution_improved(w,v,W)
        if best_V < _V:
            best_sol, best_W, best_V = sol, _W, _V
        v_all.append(_V)
        v_best.append(best_V)
    # plt.figure()
    # plt.plot(v_all)
    # plt.plot(v_best)
    # plt.show()
    return best_sol, best_W, best_V, v_all, v_best

def search_greedy(w, v, W, iters):
    best_sol, best_W, best_V = get_random_solution(w,v,W)
    v_all = [best_V]
    v_best = [best_V]
    num = w.shape[0]
    for i in range(iters):
        sol = best_sol.copy()
        #set random 0 bit to 1
        indx = np.random.randint(num)
        while sol[indx%num] == True:
            indx = indx + 1
        sol[indx%num] = True
        #correct if needed
        if w[sol].sum() > W:
            correct_solution(w,v,W,sol)
        _V = v[sol].sum()
        _W = w[sol].sum()
        if best_V < _V:
            best_sol, best_W, best_V = sol.copy(), _W, _V
        v_all.append(_V)
        v_best.append(best_V)
    # plt.figure()
    # plt.plot(v_all)
    # plt.plot(v_best)
    # plt.show()
    return best_sol, best_W, best_V, v_all, v_best

def search_greedy_improved(w, v, W, iters):
    best_sol, best_W, best_V = get_random_solution_improved(w,v,W)
    v_all = [best_V]
    v_best = [best_V]
    num = w.shape[0]
    for i in range(iters):
        sol = best_sol.copy()
        #set random 0 bit to 1
        indx = np.random.randint(num)
        while sol[indx%num] == True:
            indx = indx + 1
        sol[indx%num] = True
        #correct if needed
        if w[sol].sum() > W:
            correct_solution2(w,v,W,sol)
        _V = v[sol].sum()
        _W = w[sol].sum()
        if best_V < _V:
            best_sol, best_W, best_V = sol.copy(), _W, _V
        v_all.append(_V)
        v_best.append(best_V)
    # plt.figure()
    # plt.plot(v_all)
    # plt.plot(v_best)
    # plt.show()
    return best_sol, best_W, best_V, v_all, v_best

num = 50  # liczba przedmiotów
wmin = 1  # minimlana waga
wmax = 100  # maksymalna waga
vmin = 1  # minimalna wartosc
vmax = 100  # maksymalna wartosc
knapsack_perc = 0.5  # pojemnosc plecaka jako procent sumy wag wszystkich przedmiotow

w, v = generate_problem(wmin, wmax, vmin, vmax, num)  # w - wagi, v - wartosci
Wall = w.sum()
Vall = v.sum()
W = int(knapsack_perc * Wall) # pojemnosc plecaka

number_of_tests = 10

random_search = np.ndarray(shape=(number_of_tests, 2))
random_search_improved = np.ndarray(shape=(number_of_tests, 2))
greedy_search = np.ndarray(shape=(number_of_tests, 2))
greedy_search_improved = np.ndarray(shape=(number_of_tests, 2))

for i in range(number_of_tests):
    greedy = search_greedy(w, v, W, 1000)
    random = search_random(w, v, W, 1000)
    greedy_search[i] = [greedy[1], greedy[2]]
    random_search[i] = [random[1], random[2]]

    greedy = search_greedy_improved(w, v, W, 1000)
    random = search_random_improved(w, v, W, 1000)
    greedy_search_improved[i] = [greedy[1], greedy[2]]
    random_search_improved[i] = [random[1], random[2]]

print("Greedy search")
print('\tSuma wag:', np.average(a=greedy_search, axis=0)[0])
print('\tSuma wartosci:', np.average(a=greedy_search, axis=0)[1])

print("Greedy search improved")
print('\tSuma wag:', np.average(a=greedy_search_improved, axis=0)[0])
print('\tSuma wartosci:', np.average(a=greedy_search_improved, axis=0)[1])

print("Random search")
print('\tSuma wag:', np.average(a=random_search, axis=0)[0])
print('\tSuma wartosci:', np.average(a=random_search, axis=0)[1])

print("Random search improved")
print('\tSuma wag:', np.average(a=random_search_improved, axis=0)[0])
print('\tSuma wartosci:', np.average(a=random_search_improved, axis=0)[1])

Greedy search
	Suma wag: 1286.5
	Suma wartosci: 2119.3
Greedy search improved
	Suma wag: 1283.8
	Suma wartosci: 2146.5
Random search
	Suma wag: 1263.1
	Suma wartosci: 1681.3
Random search improved
	Suma wag: 1272.2
	Suma wartosci: 1898.0


Twoje wyniki i wnioski umieść w komórce poniżej.

Greedy search
	Suma wag: 1286.5
	Suma wartosci: 2119.3
Greedy search improved
	Suma wag: 1283.8
	Suma wartosci: 2146.5
Random search
	Suma wag: 1263.1
	Suma wartosci: 1681.3
Random search improved
	Suma wag: 1272.2
	Suma wartosci: 1898.0
    
Wnioski:
    Zdecydowaną poprawę widać w przypadku metody random search, dla greedy search różnica pomiędzy użyciem dwóch różnych funkcji naprawiających jest bardzo niewielka. Wynika to ze sposobu w jaki obydwie metody działają - random search nie wykorzystuje wiedzy o najlepszych do tej pory rozwiązaniach, dlatego napisana w taki sposób funkcja naprawiająca tak bardzo porpawia wyniki.

In [ ]:
#Komórka z autotestem

## Zadanie 3 (6 pkt.)

Dostosuj swoją implementację algorytmu genetycznego do problemu plecakowego. 

- Jakie wyniki można uzyskać z jego pomocą? Czy działa on zawsze lepiej niż inne heurystyki?

- Która procedura naprawcza działa lepiej w algorytmie genetycznym?

- Przedstaw wnioski na podstawie uśrednionych wyników dla problemów plecakowych o rozmiarze 50, 100, 300.

- Problemy plecakowe i najlepsze znalezione rozwiązania zapisz do plików.

UWAGA! Po wygenerowaniu problemu plecakowego, przed uruchomieniem algorytmu genetycznego (lub innego) wywołaj:

`np.random.seed(int(time.time()))`

tak by problem plecakowy generował się ten sam, ale algorytmy miały szansę na nowy przebieg.

In [6]:
import math
import numpy as np
import time
from mpl_toolkits.mplot3d import axes3d
import matplotlib.pyplot as plt

def correct_solution(w, v, W, sol):
    num = w.shape[0]
    while w[sol].sum() > W:
        indx = np.random.randint(num)
        while sol[indx%num] == False:
            indx = indx + 1
        sol[indx%num] = False

def correct_solution2(w, v, W, sol):
    ratio = v/w
    num = w.shape[0]
    while w[sol].sum() > W:
        while True:
            indx = np.argmin(ratio)
            if sol[indx] == False:
                ratio[indx] = 10000000
            else:
                sol[indx] = False
                break

def get_random_solution(w, v, W):
    num = w.shape[0]
    sol = np.random.randint(0,2, size=num, dtype=np.bool)  # 1 / True oznacza, ze przedmiot jest wybrany
    _V = np.sum(v[sol])
    _W = np.sum(w[sol])
    if _W > W:
        correct_solution(w,v,W,sol)
        _V = np.sum(v[sol])
        _W = np.sum(w[sol])
    return sol, _W, _V

def search_random(w,v,W,iters):
    best_sol, best_W, best_V = get_random_solution(w,v,W)
    v_all = [best_V]
    v_best = [best_V]
    for i in range(iters):
        sol, _W, _V = get_random_solution(w,v,W)
        if best_V < _V:
            best_sol, best_W, best_V = sol, _W, _V
        v_all.append(_V)
        v_best.append(best_V)
#     plt.figure()
#     plt.plot(v_all)
#     plt.plot(v_best)
#     plt.show()
    return best_sol, best_W, best_V, v_all, v_best

def search_greedy_improvement(w, v, W, iters):
    best_sol, best_W, best_V = get_random_solution(w,v,W)
    v_all = [best_V]
    v_best = [best_V]
    num = w.shape[0]
    for i in range(iters):
        sol = best_sol.copy()
        #set random 0 bit to 1
        indx = np.random.randint(num)
        while sol[indx%num] == True:
            indx = indx + 1
        sol[indx%num] = True
        #correct if needed
        if w[sol].sum() > W:
            correct_solution(w,v,W,sol)
        _V = v[sol].sum()
        _W = w[sol].sum()
        if best_V < _V:
            best_sol, best_W, best_V = sol.copy(), _W, _V
        v_all.append(_V)
        v_best.append(best_V)
#     plt.figure()
#     plt.plot(v_all)
#     plt.plot(v_best)
#     plt.show()
    return best_sol, best_W, best_V, v_all, v_best

#Pakuje najpierw najbardziej wartościowe przedmioty
def get_value_first(w, v, W):
    ii = np.argsort(-v)
    num = w.shape[0]
    sol = np.repeat(False, num)
    _W = 0
    for i in range(num):
        if _W + w[ii[i]] <= W:
            sol[ii[i]] = True
            _W = _W + w[ii[i]]
    _V = v[sol].sum()
    return sol, _W, _V

def get_ratio_first(w, v, W):
    ii = np.argsort(-v/w) #stosunek wartosci do wagi
    num = w.shape[0]
    sol = np.repeat(False, num)
    _W = 0
    for i in range(num):
        if _W + w[ii[i]] <= W:
            sol[ii[i]] = True
            _W = _W + w[ii[i]]
    _V = v[sol].sum()
    return sol, _W, _V

def generate_problem(wmin, wmax, vmin, vmax, items_num):
    w = np.random.randint(wmin, wmax, size=items_num)  #weight
    v = np.random.randint(vmin, vmax, size=items_num)  #values
    return w, v

def gen_pop(w, v, W, pop_size, corr_method = 1):
    '''
    Method for generating population.
    Input:
        w - list of weights of items in knapsack
        v - list of values of items in knapsack
        W - capacity of knapsack
        pop_size - size of generated population should be
        corr_method - index of correction method (1 - standard method, 2 - improved)
    '''
    population = np.random.randint(0, 2, size=(pop_size, len(w)), dtype=bool)
    for individual in population:
        if corr_method == 1: # Basic, random method
            individual = correct_solution(w, v, W, individual)
        elif corr_method == 2: # Custom, improved method
            individual = correct_solution2(w, v, W, individual)

    return population

def evaluate(population, v):
    '''
    Method for evaluating population.
    :param population: population generated before
    :param v: list of values of items in knapsack
    '''
    evaluated_pop = np.ndarray(shape = (len(population),))
    for i in range(len(evaluated_pop)):
        evaluated_pop[i] = np.sum(v[population[i]])
    return evaluated_pop

def select(pop, evaluated_pop):
    '''
    Method for selection from population
    :param pop: Population
    :param evaluated_pop:
    :return:
    '''
    new_pop = np.copy(pop)
    sum = evaluated_pop.sum()
    probability = np.ndarray(shape=len(evaluated_pop), dtype="float")
    probability[0] = evaluated_pop[0]/sum
    for i in range(len(evaluated_pop)-1):
        probability[i+1] = evaluated_pop[i+1] / sum + probability[i]
    for i in range(len(evaluated_pop)):
        rand = np.random.random()
        for j in range(len(evaluated_pop)):
            if probability[j] > rand:
                new_pop[i] = pop[j]
                break
    return new_pop


def xover(pop, p, w, v, W, corr_method = 1):
    '''
    Method for crossing given population.
    :param pop: Given population
    :param p: Probability of crossing
    :param w: List of weights of items in knapsack
    :param v: List of values of items in knapsack
    :param W: Capacity of knapsack
    :param corr_method: index of correction method (1 - standard method, 2 - improved)
    :return: Crossed population.
    '''
    new_pop = np.copy(pop)
    pop_len = len(pop)
    for i in range(0, pop_len - 1, 2):
        point = int(np.random.rand() * (len(pop[i]) - 1))
        p1 = np.concatenate((pop[i][0:point], pop[i + 1][point:]))
        p2 = np.concatenate((pop[i + 1][0:point], pop[i][point:]))

        new_pop[i] = np.array(p1)
        new_pop[i + 1] = np.array(p2)

    for i in new_pop:
        if corr_method == 1:
            i = correct_solution(w, v, W, i)
        elif corr_method == 2:
            i = correct_solution2(w, v, W, i)
    return new_pop


def mutate(pop, pm, w, v, W, corr_method = 1):
    '''
    Method for mutating given population.
    :param pop: Given population
    :param pm: Probability of mutation
    :param w: List of weights of items in knapsack
    :param v: List of values of items in knapsack
    :param W: Capacity of knapsack
    :param corr_method: index of correction method (1 - standard method, 2 - improved)
    :return: Mutated population
    '''
    for i in range(len(pop)):
        for j in range(len(pop[i])):
            if np.random.rand() <= pm:
                pop[i][j] = not pop[i][j]

        if corr_method == 1:
            pop[i] = correct_solution(w, v, W, pop[i])
        elif corr_method == 2:
            pop[i] = correct_solution2(w, v, W, pop[i])
    return pop

def evolve_knapsack(w, v, W, pop_size, pxover, pmutate, generations, corr_method):
    pop = gen_pop(w, v, W, pop_size, corr_method)
    evals = evaluate(pop, v)
    i = np.argmax(evals)
    best = pop[i].copy()
    best_V = evals[i]
    best_iter = 0
    v_all = [best_V]
    v_best = [best_V]
    v_mean = [np.mean(evals)]

    # print('initial best', best_V)

    for i in range(generations):
        pop = select(pop, evals)
        pop = xover(pop, pxover, w, v, W, corr_method)
        pop = mutate(pop, pmutate, w, v, W, corr_method)
        evals = evaluate(pop, v)
        ii = np.argmax(evals)
        temp_best_v = evals[ii]
        if temp_best_v > best_V:
            best_V = temp_best_v
            best_iter = i + 1
            best = pop[ii].copy()
            print('better solution of ', best_V, 'in', best_iter)
        v_all.append(temp_best_v)
        v_best.append(best_V)
        v_mean.append(np.mean(evals))

    return best, w[best].sum(), best_V


def genetic_test(number_of_tests, w, v, W, pop_size, pxover, pmutate, generations, corr_method):
    w_best = np.ndarray(shape=(number_of_tests,))
    v_best = np.ndarray(shape=(number_of_tests,))
    p_best = np.ndarray(shape=(number_of_tests, len(w)), dtype="bool")

    for t in range(number_of_tests):
        best, best_w, best_v = evolve_knapsack(w, v, W, pop_size, pxover, pmutate, generations, corr_method)
        p_best[t] = best
        w_best[t] = best_w
        v_best[t] = best_v

    max = np.argmax(v_best)
    best_pop = p_best[max]
    return np.average(w_best), np.average(v_best), best_pop


def genetic_test_complete(knapsack_size):
    # Setting custom seed
    np.random.seed(125689)

    w, v = generate_problem(1, 100, 1, 100, knapsack_size)  # w - wagi, v - wartosci
    kn_weight = w.sum()
    kn_value = v.sum()
    W = int(0.5 * kn_weight)
    print("Rozmiar problemu:", knapsack_size)

    print('\tPojemnosc:', W)
    print('\tWagi:', w)
    print("\tSuma wag:", kn_weight)
    print('\tWartosci:', v)
    print("\tSuma wartosci:", kn_value)

    # Setting seed back to random
    np.random.seed(int(time.time()))

    pop_size = 50
    pxover = 0.9
    pmutate = 0.02
    generations = 100
    number_of_tests = 20

    print("\n\nAlgorytm genetyczny")

    # Basic correction method
    w_basic, v_basic, best_pop_basic = genetic_test(number_of_tests, w, v, W, pop_size, pxover, pmutate, generations, 1)
    print("Podstawowa metoda naprawcza:")
    np.savetxt('Genetic - basic correction method, size ' + str(knapsack_size) + ".txt", best_pop_basic, delimiter=',')

    print("\tSrednia waga:", w_basic)
    print("\tSrednia wartosc:", v_basic)

    # Custom correction method
    w_custom, v_custom, best_pop_custom = genetic_test(number_of_tests, w, v, W, pop_size, pxover, pmutate, generations, 2)
    print("Ulepszona metoda naprawcza:")
    np.savetxt('Genetic - custion correction method, size ' + str(knapsack_size) + ".out", best_pop_custom, delimiter=',')

    print("\tSrednia waga:", w_custom)
    print("\tSrednia wartosc:", v_custom)


def heuristics_tests(knapsack_size):
    # Setting custom seed
    np.random.seed(125689)

    w, v = generate_problem(1, 100, 1, 100, knapsack_size)  # w - wagi, v - wartosci
    kn_weight = w.sum()
    kn_value = v.sum()
    W = int(0.5 * kn_weight)
    print("Rozmiar problemu:", knapsack_size)

    number_of_tests = 20

    # Setting seed back to random
    np.random.seed(int(time.time()))

    print("Random search:")
    w_best = np.ndarray(shape=(number_of_tests,), dtype="float")
    v_best = np.ndarray(shape=(number_of_tests,), dtype="float")
    for i in range(number_of_tests):
        sol_random_search = search_random(w, v, W, 1000)
        w_best[i] = sol_random_search[1]
        v_best[i] = sol_random_search[2]

    print('\tSrednia waga:', np.average(w_best))
    print('\tSrednia wartosc:', np.average(v_best))

    print("Greedy search:")
    w_best = np.ndarray(shape=(number_of_tests,), dtype="float")
    v_best = np.ndarray(shape=(number_of_tests,), dtype="float")
    for i in range(number_of_tests):
        sol_greedy = search_greedy_improvement(w, v, W, 1000)
        w_best[i] = sol_greedy[1]
        v_best[i] = sol_greedy[2]

    print('\tSrednia waga:', np.average(w_best))
    print('\tSrednia wartosc:', np.average(v_best))

    print("Value first:")
    w_best = np.ndarray(shape=(number_of_tests,), dtype="float")
    v_best = np.ndarray(shape=(number_of_tests,), dtype="float")
    for i in range(number_of_tests):
        sol_value_first = get_value_first(w, v, W)
        w_best[i] = sol_value_first[1]
        v_best[i] = sol_value_first[2]

    print('\tSrednia waga:', np.average(w_best))
    print('\tSrednia wartosc:', np.average(v_best))

    print("Ratio first:")
    w_best = np.ndarray(shape=(number_of_tests,), dtype="float")
    v_best = np.ndarray(shape=(number_of_tests,), dtype="float")
    for i in range(number_of_tests):
        sol_ratio_first = get_ratio_first(w, v, W)
        w_best[i] = sol_ratio_first[1]
        v_best[i] = sol_ratio_first[2]

    print('\tSrednia waga:', np.average(w_best))
    print('\tSrednia wartosc:', np.average(v_best))
    print("\n")

def tests():
    genetic_test_complete(50)
    heuristics_tests(50)
    genetic_test_complete(100)
    heuristics_tests(100)
    genetic_test_complete(300)
    heuristics_tests(300)

tests()

Rozmiar problemu: 50
	Pojemnosc: 1119
	Wagi: [25 33 33 37 70 20 55 80 21 90 20 47 96 12 68 82 11 74 38 27 12 66 96 36
 19  6 29 71  2 25 58 53 40 89 52 24 11 65 26 47 45 42 49 58 33 17 95 40
 61 33]
	Suma wag: 2239
	Wartosci: [81 52 22 88 20 38 76 54 31 59 89  3 88 57 73 57 82 23 74 57 75 28 95 15
 34 68 65 21 14 29  4  6 91 88 13 25 77 51 14 47 99 34 84 84 37 12 86 84
 72 17]
	Suma wartosci: 2593


Algorytm genetyczny


F:\ProgramFiles\Anaconda\envs\wdsienv\lib\site-packages\ipykernel_launcher.py:151: RuntimeWarning: invalid value encountered in double_scalars
F:\ProgramFiles\Anaconda\envs\wdsienv\lib\site-packages\ipykernel_launcher.py:153: RuntimeWarning: invalid value encountered in double_scalars


Podstawowa metoda naprawcza:
	Srednia waga: 1081.8
	Srednia wartosc: 1509.95
Ulepszona metoda naprawcza:
	Srednia waga: 1101.3
	Srednia wartosc: 1686.8
Rozmiar problemu: 50
Random search:
	Srednia waga: 1091.55
	Srednia wartosc: 1651.4
Greedy search:
	Srednia waga: 1113.1
	Srednia wartosc: 1970.7
Value first:
	Srednia waga: 1112.0
	Srednia wartosc: 1906.0
Ratio first:
	Srednia waga: 1112.0
	Srednia wartosc: 2012.0


Rozmiar problemu: 100
	Pojemnosc: 2416
	Wagi: [25 33 33 37 70 20 55 80 21 90 20 47 96 12 68 82 11 74 38 27 12 66 96 36
 19  6 29 71  2 25 58 53 40 89 52 24 11 65 26 47 45 42 49 58 33 17 95 40
 61 33 81 52 22 88 20 38 76 54 31 59 89  3 88 57 73 57 82 23 74 57 75 28
 95 15 34 68 65 21 14 29  4  6 91 88 13 25 77 51 14 47 99 34 84 84 37 12
 86 84 72 17]
	Suma wag: 4832
	Wartosci: [72 65 28 76 31 93 84 21 96  6 86 97 32 10 20 64 61 82 58  4 22 56 13 72
 59 36 71 74 91 69 19 92 26 39 21 80 45 41 48 86 47 50 87 52 86 88 10  2
 75 76 58 64 82 58 91 56 81 62 10  4 55  1  5 38 20 68 

Twoje wyniki i wnioski umieść w komórce poniżej.

### Wnioski

1. Jakie wyniki można uzyskać z pomocą algorytmu genetycznego? Czy działa on zawsze lepiej niż inne heurystyki?
    - Algorytm genetyczny nie działa lepiej niż inne heurystyki. Wyniki uzyskane z jego pomocą (przu użyciu ulepszonej procedury naprawczej) są porównywalne tylko z metodą random search.
2. Która procedura naprawcza działa lepiej w algorytmie genetycznym?
    - Lepiej działa procedura naprawcza napisana przeze mnie, poprawa wyników jest zauważalna.
3. Przedstaw wnioski na podstawie uśrednionych wyników dla problemów plecakowych o rozmiarze 50, 100, 300.
    - Im większy problem plecakowy, tym bardziej widoczna staje się przewaga innych heurystyk względem algorytmu genetycznego. Osiągają one o wiele lepsze wyniki (z wyjątkiem random search, która uzyskuje wyniki porównywalne), dodatkowo wykonując się o wiele szybciej - przy problemie plecakowym o rozmiarze 300 algorytm genetyczny wykonywał się kilkanaście sekund, podczas gdy wszystkie pozostałe metody zakończyły obliczenia w około 1-2 sekund.
    
    
### Wyniki
1. Rozmiar problemu: 50, pojemnosc: 1119
    1. Algorytm genetyczny
        1. Podstawowa metoda naprawcza:
            - Srednia waga: 1081.8
            - Srednia wartosc: 1509.95
        2. Ulepszona metoda naprawcza:
            - Srednia waga: 1101.3
            - Srednia wartosc: 1686.8
    2. Random search:
        - Srednia waga: 1091.55
        - Srednia wartosc: 1651.4
    3. Greedy search:
        - Srednia waga: 1113.1
        - Srednia wartosc: 1970.7
    4. Value first:
        - Srednia waga: 1112.0
        - Srednia wartosc: 1906.0
    5. Ratio first:
        - Srednia waga: 1112.0
        - Srednia wartosc: 2012.0
        
2. Rozmiar problemu: 100, pojemnosc: 2416
    1. Algorytm genetyczny
        1. Podstawowa metoda naprawcza:
            - Srednia waga: 2349.4
            - Srednia wartosc: 2885.3
        2. Ulepszona metoda naprawcza:
            - Srednia waga: 2376.45
            - Srednia wartosc: 3108.35
    2. Random search:
        - Srednia waga: 2367.55
        - Srednia wartosc: 3091.45
    3. Greedy search:
        - Srednia waga: 2408.65
        - Srednia wartosc: 4062.8
    4. Value first:
        - Srednia waga: 2415.0
        - Srednia wartosc: 3891.0
    5. Ratio first:
        - Srednia waga: 2415.0
        - Srednia wartosc: 4174.0
        
3. Rozmiar problemu: 300, pojemnosc: 7531
    1. Algorytm genetyczny
        1. Podstawowa metoda naprawcza:
            - Srednia waga: 7439.85
            - Srednia wartosc: 7867.7
        2. Ulepszona metoda naprawcza:
            - Srednia waga: 7487.95
            - Srednia wartosc: 8338.35
    2. Random search:
        - Srednia waga: 7493.1
        - Srednia wartosc: 8252.45
    3. Greedy search:
        - Srednia waga: 7524.55
        - Srednia wartosc: 11222.45
    4. Value first:
        - Srednia waga: 7531.0
        - Srednia wartosc: 11284.0
    5. Ratio first:
        - Srednia waga: 7527.0
        - Srednia wartosc: 12066.0


<div style="text-align: right">&copy; Zakład Inteligencji Obliczeniowej, Instytut Informatyki, Politechnika Krakowska </div>